# 4 - Influential Classification Models (and Tools)

## Introduction

This Notebook (4) will continue on from the previous sections. This notebook will go through the process of using __Keras Applications and reusing model__. Previously, the notebook focused on implementing the ResNet model from scratch, here, the intention is to utilise the models that were curated from Keras Applications and be used as direct comparison with the self-implemented version.

## Supporting Utilities .py files:

In this Notebook, there will be a requirement to import the code/utilities from the following files (.py files):
- DataPrepCIFAR_utility.py

## Dataset:

For this part of the project, the CIFAR-100 dataset will be used, it is a collection of of 60,000 32x32 images that have 100 classes. CIFAR-100 was originally collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. It iss also a subset of the 80 million tiny images dataset. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses.

Source: https://www.cs.toronto.edu/~kriz/cifar.html

Further, the TensorFlow team offers a python package called "tensorflow_datasets" that provides the helper function to download tthis dataset as well as other more common ones. For the purposes of this project, the CIFAR-100 dataset will be download with this package.

Source: https://www.tensorflow.org/datasets/catalog/cifar100

## Requirements:
- Tensorflow 2.0 (GPU is better)
- Tensorflow-Hub
- Keras (GPU is better)

### Import the required libraries:

In [1]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import math
import timeit

In [2]:
import os
# from IPython.display import display, Image
# import matplotlib.pyplot as plt

# %matplotlib inline

# Set up the working directory for the images:
image_folderName = 'Description Images'
image_path = os.path.abspath(image_folderName) + '/'

In [3]:
# Set the random set seed number: for reproducibility.
Seed_nb = 42

### GPU Information:

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
devices = sess.list_devices()
devices

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5



[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 763406161721560341),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6586313605, 2607108048413981961)]

## 1 - Hyperparameters:

In [5]:
input_shape = [224, 224, 3]
batch_size = 64
nb_epochs = 300

## 2 - Data Preparation and Defining the Input Pipeline:

To use the TensorFlow package, please ensure to install it: "pip install tensorflow-datasets" or use the Anaconda Navigator.

## 2.1 - Download the Dataset:

In [6]:
import tensorflow_datasets as tfds
import DataPrepCIFAR_utility

In [7]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [8]:
# Get the Dataset Information:
cifarData_info = DataPrepCIFAR_utility.get_info()
print(cifarData_info)

tfds.core.DatasetInfo(
    name='cifar100',
    version=1.3.1,
    description='This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).',
    urls=['https://www.cs.toronto.edu/~kriz/cifar.html'],
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=tf.int64, num_classes=20),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=100),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multi

## 2.2 - Set up the dataset:

In [9]:
# Set the number of classes:
nb_classes = cifarData_info.features['label'].num_classes

# Define the number of images for training and validation:
nb_train_imgs = cifarData_info.splits['train'].num_examples
nb_valid_imgs = cifarData_info.splits['test'].num_examples

## 2.3 - Set the "steps_per_epoch" for Training and Validation Set:

From the previous code block, the dataset is compatible with Keras methods liike "model.fit()", but will also require the "steps_per_epoch" to be specified. This is the number of batches per epoch of the Dataset objects that has to be specified for the Keras method so that it can work properly together. This is done for the training batches ("steps_per_epoch") and validation batches ("validation_steps").

In [10]:
train_steps_per_epoch = math.ceil(nb_train_imgs / batch_size)

valid_steps_per_epoch = math.ceil(nb_valid_imgs / batch_size)

Now that the CIFAR-100 dataset has been prepared. The next stage is to call the ResNet model.

## 3 - Grab the ResNet-50 model from Keras Applications:

Keras Applications consists of many well-curated models. Keras Applications are deep learning models that are made available from the team and it also contains the pre-trained weights for each of these model. These models can be immediately used for prediction, feature extraction, and fine-tuning. The model that will be instantiated here will be the ResNet-50 model. 

## 3.1 - Instatiate the ResNet-50 model:

Like the previous notebooks, the image sizes for the CIFAR-100 are to be set as 224x224 pixel sizes, as suggested by the creators of ResNet.

In [11]:
# Instantiate the model:
resnet50 = tf.keras.applications.resnet50.ResNet50(include_top=True,
                                                   weights= None,
                                                   input_shape= input_shape,
                                                   classes= nb_classes
                                                  )

# Model Summary:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## 3.2 - Comparison to the Self-implemented model:

Self-implement ResNet model have the following specifications:
- Total params: 23,777,252
- Trainable params: 23,731,812
- Non-trainable params: 45,440

Keras Applications ResNet model have the following specifications:
- Total params: 23,792,612
- Trainable params: 23,739,492
- Non-trainable params: 53,120

As it can be seen, the total number of parameters are slightly different. To find out more to why this is, a more detailed comparison of the code between the two models will be required. The link to the official implementation is: https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py

## 4 - Training the ResNet-50 Model:

With the ResNet model instantiated, the model can now be trained on the dataset.



## Summary:

Although this is not the end of this project, it does conclude the 1st notebook relevant to the theory of these advanced deep learning models. Please go to check out __Notebook 5__ for the __Getting Models from TensorFlow Hub__.